In [7]:
import os, sys
from collections import defaultdict

import numpy as np
import pandas as pd
import pickle

current_wd = '/home/mexposit/LMPM/lmpm'

from lmpm.unirep import get_UniReps
from lmpm.check_inputs import check_input

In [9]:

# get current path to load files as relative paths later
# using os.path.join to avoid issues across OS using different backslash symbols
# Importing dG parameters to params_dict
params = pd.read_csv(os.path.join(current_wd,"models","Params.csv"), header=None)
lst = params[0].values
params.set_index(lst, inplace=True)
del params[0]
params_dict = defaultdict(list)
for i, row in params.iterrows():
    for position in range(1, len(row) + 1):
        params_dict[i].append(float(row.loc[position]))

# Load trained models
model_human = pickle.load(open(os.path.join(current_wd,"models","human.pkl"), 'rb'))
model_yeast = pickle.load(open(os.path.join(current_wd,"models","yeast.pkl"), 'rb'))
model_ecoli = pickle.load(open(os.path.join(current_wd,"models","ecoli.pkl"), 'rb'))




def model_organism_selection(organism, include_dg):
    """
    Imports trained model of secretion scores based on organism data set

    Args:
        organism (str): specific organism of sequence type 
                        (all, human, yeast, ecoli)
        include_dg (Boolean): specify inclusion of additional features
                                (default=False)

    Returns:
        model: trained model of secretion probabilities for organism
    """
    # Create dictionary of loaded, trained models
    # Check specification of additional features
    if include_dg:
        model_dict = {
            'human': model_human,
            'yeast': model_yeast,
            'ecoli': model_ecoli
        }
    else: 
        model_dict = {
            'human': model_human,
            'yeast': model_yeast,
            'ecoli': model_ecoli
        }

    # Access the specific organism model
    model = model_dict[organism]

    return model

def calculate_transmembrane_dg(seq):
    """
    Calculates the free energy of transmembrane insertion of the sequence

    Args:
        sequence (str): amino acid sequence in single-letter format

    Returns:
        transmembrane_dG (float): free energy calculation
    """
    # First, make sure the sequence is longer than 19 residues
    if len(seq) < 19:
        raise ValueError('dG cannot be computed on sequences \
                            less than 19 residues')

    # Next, check if there are any ambiguous amino acids
    # also convert to appropiate format if it was provided in 3 letter code
    sequence = check_input(seq)

    # Starts dG calculations
    dg_values = []

    # Scans the sequence for every 19-residue frame
    for j in range(len(sequence)-18):
        running_total = 0
        segment = sequence[j:j+19]
        for k, amino_acid in enumerate(segment):
            running_total += params_dict[amino_acid][k]
        dg_values.append(running_total)

    # Finds the minimum dG value
    transmembrane_dG = min(dg_values)

    return transmembrane_dG

/home/mexposit/miniconda3/envs/lmpmdev/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/mexposit/miniconda3/envs/lmpmdev/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [195]:
def predict_loc_simple(input_seq, organism, target_loc, include_dg=False):
    
    ## Tests to check inputs
    # target class should be one of the classes of the model
    if target_loc not in ['secreted', 'membrane', 'cytoplasm']:
        raise ValueError('The selected probability was: "'+str(target_loc)+'", but should be either secreted, membrane or cytoplasm')
    else:
        pass
    
    # Obtain unirep representation of sequence
    sequence = check_input(input_seq)
    values = get_UniReps(sequence)[0] # do we need this [0]?
    if include_dg:
        dg = calculate_transmembrane_dg(sequence)
        dg_standardized = (dg - 2.053619) / (2.300432 ** 2)
        values = np.append(values, dg_standardized)
    values = values.reshape(1, -1)
    
    # Select the model based on the organism
    model = model_organism_selection(organism, include_dg)
    # Grab the probabilities from the model
    classes = list(model.classes_)
    prediction_probabilities = list(model.predict_proba(values)[0])
    
    pred = prediction_probabilities[classes.index(target_loc)]

    return pred

In [198]:
def predict_location(input_seq, organism=None, target_loc=None, include_dg=False, pred_all=False):
    """
    Finds the probability of being the target class given the
    sequence based on model and UniRep representation only

    Args:
        input_seq (str): sequence entered by the user
        organism (str):  specific organism of sequence type 
                         (all, human, yeast, ecoli)
        target_loc (str): class to predict the probability
                            (secreted (default), cytoplasm, or membrane)
        include_dg (Boolean): specify inclusion of additional features
                                (default=False)

    Returns:
        predicted_loc (str): predicted localization of the protein
                        (cytoplasm, membrane, secreted)
        target_score (float): probability of sequence being 
                            in target class
    """
    
    ## Tests to check inputs
    # target class should be one of the classes of the model
    if target_loc not in ['secreted', 'membrane', 'cytoplasm','all']:
        raise ValueError('The selected probability was: "'+str(target_loc)+'", but should be either secreted, membrane, cytoplasm or all(default)')
    else:
        pass
    
    # Keep query
    query = {'sequence': input_seq,
             'organism': organism,
             'location': target_loc,
             'include_dg': include_dg
    }
    
    
    # Obtain unirep representation of sequence
    sequence = check_input(input_seq)
    values = get_UniReps(sequence)[0] # do we need this [0]?
    if include_dg:
        dg = calculate_transmembrane_dg(sequence)
        dg_standardized = (dg - 2.053619) / (2.300432 ** 2)
        values= np.append(values, dg_standardized)
    values = values.reshape(1, -1)
    
    # process query
    # if user decided to predict for all organisms, get them
    if organism == 'all' or pred_all:
        pred_org = ['human', 'yeast', 'ecoli']
    else:
        pred_org = [organism]

    pred_results = pd.DataFrame()
    
    # make predictions for each organism
    for org in pred_org:
        # Select the model based on the organism
        model = model_organism_selection(org, include_dg)
        # Grab the probabilities from the model
        classes = list(model.classes_)
        prediction_probabilities = list(model.predict_proba(values)[0])
        pred_results[org] = pd.Series(prediction_probabilities, index=classes)
    
    # if user decided to predict all but return results for only one
    if pred_all and organism != 'all':
        filt_results = pred_results.loc[:,organism]
    else:
        filt_results = pred_results
    
    # filter results accordingly
    if target_loc == 'all':
        filt_results = filt_results
    else:
        filt_results = filt_results.loc[target_loc]
    
    # get the best prediction
    if organism == 'all':
        org_for_pred = 'human'
    else:
        org_for_pred = organism
    
    predicted_loc = pred_results.index[np.argmax(pred_results.loc[:,org_for_pred])]
    predicted_prob = np.max(pred_results.loc[:,org_for_pred])
    
    if pred_all:
        all_predictions = pred_results
    else:
        all_predictions = 'not computed'
    
    class PredLocalization(object):
        def __init__(self):
            self.query = query
            self.result = filt_results
            self.predicted_loc = predicted_loc
            self.predicted_prob = predicted_prob
            self.all_predictions = all_predictions
            
        def __call__(self):
            return filt_results

    return PredLocalization()


In [210]:
preds = predict_location('AWYAWYAWYLAWYLAWYLAWYL','all','secreted')

In [213]:
preds()

human    0.17
yeast    0.07
ecoli    0.05
Name: secreted, dtype: float64

In [214]:
# same as preds()
preds.result

human    0.17
yeast    0.07
ecoli    0.05
Name: secreted, dtype: float64

In [215]:
# when organism = all, equals to the human prediction
preds.predicted_loc

'membrane'

In [216]:
preds.predicted_prob

0.58

In [217]:
preds.query

{'sequence': 'AWYAWYAWYLAWYLAWYLAWYL',
 'organism': 'all',
 'location': 'secreted',
 'include_dg': False}

In [218]:
preds.all_predictions

'not computed'

In [219]:
preds.predicted_loc

'membrane'

In [220]:
predict_loc_simple('AWYAWYAWYLAWYLAWYLAWYL','human','cytoplasm')

0.25

In [221]:
preds = predict_location('AWYAWYAWYLAWYLAWYLAWYL','all','all')

In [222]:
preds()

,human,yeast,ecoli
cytoplasm,0.25,0.50,0.30
membrane,0.58,0.43,0.65
secreted,0.17,0.07,0.05


In [223]:
preds = predict_location('AWYAWYAWYLAWYLAWYLAWYL','ecoli','all',False,True)

In [224]:
preds()

cytoplasm    0.30
membrane     0.65
secreted     0.05
Name: ecoli, dtype: float64

In [225]:
preds.predicted_loc

'membrane'

In [226]:
preds.predicted_prob

0.65

In [227]:
preds.all_predictions

,human,yeast,ecoli
cytoplasm,0.25,0.50,0.30
membrane,0.58,0.43,0.65
secreted,0.17,0.07,0.05


In [228]:
preds = predict_location('AWYAWYAWYLAWYLAWYLAWYL','ecoli','all',True,True)

In [229]:
preds.result

cytoplasm    0.30
membrane     0.65
secreted     0.05
Name: ecoli, dtype: float64

In [230]:
preds.all_predictions

,human,yeast,ecoli
cytoplasm,0.25,0.50,0.30
membrane,0.58,0.43,0.65
secreted,0.17,0.07,0.05


In [231]:
preds = predict_location('AYLAYLAYLAYLAYLAYLAYLAYL','ecoli','secreted',True,True)

In [232]:
preds()

0.05

In [233]:
preds.all_predictions

,human,yeast,ecoli
cytoplasm,0.24,0.39,0.18
membrane,0.56,0.58,0.77
secreted,0.20,0.03,0.05


In [234]:
preds.query['location']

'secreted'

In [235]:
preds.query

{'sequence': 'AYLAYLAYLAYLAYLAYLAYLAYL',
 'organism': 'ecoli',
 'location': 'secreted',
 'include_dg': True}

In [236]:
preds.predicted_loc

'membrane'